### Generating Synthetic Data using Sudoku

#### Create a new sudoku game

In [34]:
from utils.sudoku import SudokuEnvironment

game = SudokuEnvironment()
game.create_puzzle(difficulty='easy')
_ = game.display()

+-------+-------+
| 1 2 | _ 4 |
| _ 4 | 1 2 |
+-------+-------+
| 2 _ | _ 3 |
| 4 _ | 2 _ |
+-------+-------+


#### Build Data generation pipeline

##### Build question

In [35]:
import random
import copy
from random import randint

def randomize_dict_values(dict_list, seed=None):
    """
    Replace each 'value' with a random different value from range 1-4.
    The new value is guaranteed to be different from the original.
    All other keys remain unchanged.

    Args:
        dict_list: List of dictionaries containing 'value' key
        seed: Random seed for reproducibility (optional)

    Returns:
        List of dictionaries with 'value' key randomized to different values
    """
    if not dict_list:
        return []

    if seed is not None:
        random.seed(seed)

    # Create deep copy to avoid modifying the original
    randomized_list = copy.deepcopy(dict_list)

    # Replace each value with a random different value
    for new_dict in randomized_list:
        original_value = new_dict['value']

        # Create possible values (1-4) excluding the original value
        possible_values = [v for v in range(1, 5) if v != original_value]

        # Randomly select from the remaining values
        new_dict['value'] = random.choice(possible_values)

    return randomized_list

def generate_answer_options(correct_deduction, wrong_deductions):
    """
    Generate multiple-choice answer options with one correct and several wrong answers.

    Args:
        correct_deduction: Dictionary containing the correct deduction
        wrong_deductions: List of dictionaries containing wrong deductions

    Returns:
        tuple: (formatted_answer_string, correct_answer_letter)
    """
    answer_options_list = []
    answer_options_set = {'A', 'B', 'C', 'D'}

    # Select a random option for the correct answer
    correct_answer_option = random.choice(list(answer_options_set))
    answer_options_set.discard(correct_answer_option)

    # Add correct answer to the list
    correct_text = f"{correct_answer_option}) Position ({correct_deduction['row']},{correct_deduction['col']}) must be {correct_deduction['value']}"
    answer_options_list.append(correct_text)

    # print(len(wrong_deductions))
    for idx, wrong_deduction in enumerate(wrong_deductions):
        wrong_answer_option = random.choice(list(answer_options_set))
        answer_options_set.discard(wrong_answer_option)
        # print(wrong_deduction)
        wrong_text = f"{wrong_answer_option}) Position ({wrong_deduction['row']},{wrong_deduction['col']}) must be {wrong_deduction['value']}"
        answer_options_list.append(wrong_text)
        if idx == 2:
            break

    # Sort by the option letter and join into a string
    answer_options_list.sort(key=lambda x: x[0])
    formatted_answers = "\n".join(answer_options_list)

    # Extract the correct answer letter (without the closing parenthesis)
    correct_letter = correct_answer_option[0]

    return formatted_answers, correct_letter

In [36]:
# 1. First we define the schema for the output data

puzzle = {
    "question": None,
    "question_parsing": None,
    "answer": None,
    "id": None,
    "sel_idx": None,
    "cot": None,
    "cot_parsing": None
}

# 2. Let's create the questions with
# define overall question
question = 'You have the following Sudoku puzzle which of the following answers is correct? \n'

# Append the game to the question
question += game.display(print_game=False)

deductions_data = game.get_explicit_deductions()[-1]
correct_deduction = deductions_data[0]
wrong_deductions = randomize_dict_values(deductions_data[1:])
opts, answer = generate_answer_options(correct_deduction, wrong_deductions)
question += "\n" + opts
print(question)

puzzle['question'] = question
puzzle['answer'] = answer

# Get permutations -> create a questions for each of the empty fields -> for possible answers a, b, c, d create one correct answer and 3 wrong answers


# 3. Let's create the question parsing
qp_raw = game.get_all_units_as_strings()
qp_string = 'The puzzle has the rows: ' + str(qp_raw['rows'])
qp_string += '\nThe puzzle has the columns: ' + str(qp_raw['columns'])
qp_string += '\nThe puzzle has the squares: ' + str(qp_raw['squares'])

print(qp_string)

puzzle['question_parsing'] = qp_string

# 4. Generating cot
# Get full deduction
cot = game.get_explicit_deductions()[0]
cot_string = "; ".join(cot)
puzzle['cot'] = cot_string

# 5. Generating cot parsing
# Get statements from deduction
statements = game.get_explicit_deductions()[1]
# get evidence from deduction
evidence = game.get_explicit_deductions()[2]

# Add verification -> wrong verifications?
puzzle['cot_parsing'] = [{"statement": statements[idx], "evidence": evidence[idx],"Verification": "true"} for idx in range(len(statements))]


# 6. Generate ids

# add random ids
rand =  randint(0, 100000000)
puzzle['id'] = rand
puzzle['sel_idx'] = rand
print("--------")

puzzle

You have the following Sudoku puzzle which of the following answers is correct? 
+-------+-------+
| 1 2 | _ 4 |
| _ 4 | 1 2 |
+-------+-------+
| 2 _ | _ 3 |
| 4 _ | 2 _ |
+-------+-------+
A) Position (0,2) must be 3
B) Position (2,2) must be 2
C) Position (2,1) must be 2
D) Position (1,0) must be 2
The puzzle has the rows: ['1 2 _ 4', '_ 4 1 2', '2 _ _ 3', '4 _ 2 _']
The puzzle has the columns: ['1 _ 2 4', '2 4 _ _', '_ 1 _ 2', '4 2 3 _']
The puzzle has the squares: [' 12_4', ' _412', ' 2_4_', ' _32_']
--------


{'question': 'You have the following Sudoku puzzle which of the following answers is correct? \n+-------+-------+\n| 1 2 | _ 4 |\n| _ 4 | 1 2 |\n+-------+-------+\n| 2 _ | _ 3 |\n| 4 _ | 2 _ |\n+-------+-------+\nA) Position (0,2) must be 3\nB) Position (2,2) must be 2\nC) Position (2,1) must be 2\nD) Position (1,0) must be 2',
 'question_parsing': "The puzzle has the rows: ['1 2 _ 4', '_ 4 1 2', '2 _ _ 3', '4 _ 2 _']\nThe puzzle has the columns: ['1 _ 2 4', '2 4 _ _', '_ 1 _ 2', '4 2 3 _']\nThe puzzle has the squares: [' 12_4', ' _412', ' 2_4_', ' _32_']",
 'answer': 'A',
 'id': 68541209,
 'sel_idx': 68541209,
 'cot': "Position (0,2) must be 3 (only valid value  because 10 - 1 - 2 - 4 = 3); Position (1,0) must be 3 (only valid value  because 10 - 1 - 2 - 4 = 3); Position (2,1) must be 1 (only valid value  because 10 - 2 - 3 - 4 = 1); Position (2,2) must be 4 (only valid value  because 10 - 1 - 2 - 3 = 4); Position (3,3) must be 1 (only valid value  because 10 - 2 - 3 - 4 = 1); Positio